<a href="https://colab.research.google.com/github/CherpanovNazim/learn-llm/blob/main/notebooks/00_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* [GitHub Repo](https://github.com/CherpanovNazim/learn-llm)

In [6]:
# wait ~3 min for installations
%%time

!pip install -qU openai==1.40.3 vllm==0.5.4 transformers==4.44.0

CPU times: user 17.7 ms, sys: 2.86 ms, total: 20.6 ms
Wall time: 3.92 s


In [17]:
# wait ~3 min for installations
%%time

# Load the default model
DEFAULT_MODEL = {"model": "hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4", "api_base": "http://localhost:8000/v1", "api_key": "EMPTY"}

#run VLLM
!nohup vllm serve {DEFAULT_MODEL['model']} --quantization awq --max-model-len=4096 > vllm.log &
!tail -f vllm.log | grep -q "Uvicorn running" && echo "Now you can start using the model"

nohup: redirecting stderr to stdout
Now you can start using the model
CPU times: user 569 ms, sys: 54.3 ms, total: 624 ms
Wall time: 1min 37s


# Why LLAMA 3 ?

* We mainly use GPT 3.5 or GPT 4 models for production use-cases
* For this course we use LLAMA 3 8B model for educational purposes
* LLAMA 3 is smaller, but still very powerful
* We use OpenAI-compatible API for our LLAMA 3 - so this should would work for GPT 3.5 or GPT 4 as well

In [18]:
# @title Explainer class
import json
import openai
from IPython.display import display, Markdown, Code, update_display
import random
from openai import OpenAI
import subprocess
import os

class Explainer:
    def __init__(self, system_prompt:str = None) -> None:
        if system_prompt is None:
            system_prompt = "You are assistant for IT specialist who doesn't know NLP."
        self.basic_system_prompt = system_prompt

        self.config = DEFAULT_MODEL

        self.client_explainer = OpenAI(base_url=self.config['api_base'], api_key = self.config['api_key'])

    def llm_call(self, prompts, additional_system_prompt=""):
        system_prompt = self.basic_system_prompt + additional_system_prompt

        completion = self.client_explainer.chat.completions.create(
            model=self.config["model"],
            temperature=0,
            messages=[{"role": "system", "content": system_prompt},
                        {"role": "user", "content": prompts}])
        print(completion)
        return completion.choices[0].message.content.strip()

    def stream_llm_call(self, prompts, additional_system_prompt=""):
        system_prompt = self.basic_system_prompt + additional_system_prompt

        completion = self.client_explainer.chat.completions.create(
            model=self.config["model"],
            temperature=0,
            stream=True,
            messages=[{"role": "system", "content": system_prompt},
                        {"role": "user", "content": prompts}])

        for chunk in completion:
            if chunk.choices[0].delta.content is not None:
                yield chunk.choices[0].delta.content

    def __call__(self, text, output_type="markdown"):
        if output_type == "markdown":
            data = list()
            display_id = display(Markdown(''), display_id=True).display_id
            for chunk in self.stream_llm_call(text, "Format your answer using Markdown."):
                data.append(chunk)
                update_display(Markdown(''.join(data)), display_id=display_id)
        elif output_type == "code":
            results = self.llm_call(text, "Provide only Python code without any side comments.")
            results = results.split("```")[1].split("```")[0]
            display(Code(results))
        else:
            raise Exception("Unknown output type. Default: markdown, code")

In [19]:
explain = Explainer()

# use this class if you want to get some explanations
# It uses LLAMA 3 to generate explanations :)
explain("difference between LLMs and other deep learning NLP models in a small table")

**Comparison of LLMs and Other Deep Learning NLP Models**

| **Model Type** | **Description** | **Key Characteristics** | **Primary Use Case** |
| --- | --- | --- | --- |
| **LLMs (Large Language Models)** | Pre-trained models that learn general language patterns and relationships | Auto-regressive architecture, self-supervised learning, massive parameter count | Conversational AI, text generation, language translation |
| **Transformers** | Encoder-decoder architecture for sequence-to-sequence tasks | Self-attention mechanism, parallel processing, fixed input sequence length | Machine translation, text summarization, question answering |
| **RNNs (Recurrent Neural Networks)** | Recurrent architecture for sequential data | Hidden state, recurrent connections, fixed sequence length | Speech recognition, sentiment analysis, language modeling |
| **CNNs (Convolutional Neural Networks)** | Convolutional architecture for grid-like data | Local connections, shared weights, fixed spatial resolution | Text classification, sentiment analysis, topic modeling |
| **BERT (Bidirectional Encoder Representations from Transformers)** | Pre-trained model for contextualized language understanding | Masked language modeling, next sentence prediction, contextualized embeddings | Question answering, sentiment analysis, text classification |

Note: This table provides a high-level comparison of the key characteristics and primary use cases of each model type. LLMs are a specific type of model that encompasses the characteristics of other models, but they are not mutually exclusive.

In [20]:
import openai
import os

# Set the base URL and API key
# For production apps it's preferable to use some secret management system and don't store the key in git repo :)

client = openai.OpenAI(
    base_url = DEFAULT_MODEL['api_base'],
    api_key = DEFAULT_MODEL['api_key']
    )

## Completion VS ChatCompletion API:  
There is also 2 variants of API or how to use these models, for example in Python: `Completion` and `ChatCompletion`.  

## [Completion API](https://platform.openai.com/docs/guides/gpt/completions-api):  
For the most common case you need to specify:
* model name
* prompt that will have the task for the model and should be a string


In [22]:
response = client.completions.create(
    model=DEFAULT_MODEL["model"],
    prompt="Write a tagline for an ice cream shop.",
    temperature=0,
    max_tokens=256,
    n=1)

In [23]:
# Example of an output as an completion object:
print(response)

Completion(id='cmpl-4272ce3ec73a4b5394727eacf59b9056', choices=[CompletionChoice(finish_reason='length', index=0, logprobs=None, text=' Here are a few ideas to get you started:\n* "Cool treats for hot days"\n* "Scoops of happiness"\n* "The cream of the crop"\n* "Chill out with us"\n* "A scoop above the rest"\n* "Sweet treats for sweet people"\n* "The coolest place in town"\n* "A taste of paradise"\n* "Indulge in the sweet life"\n* "A scoop of fun for everyone"\n* "The perfect treat for any occasion"\n* "Cool, creamy, and delicious"\n* "A sweet escape from the ordinary"\n* "The best ice cream in town, guaranteed"\n* "Savor the flavor"\n* "A scoop of joy"\n* "The sweetest spot in town"\n* "Cool treats for all seasons"\n* "A taste of heaven on earth"\n* "The ultimate ice cream experience"\n* "A scoop above the rest, every time"\n* "Sweet treats for every taste bud"\n* "The coolest ice cream shop around"\n* "A taste of pure bliss"\n* "Cool, creamy, and utterly delicious"\n* "The perfect tr

In [24]:
print(response.choices[0].text.strip())

Here are a few ideas to get you started:
* "Cool treats for hot days"
* "Scoops of happiness"
* "The cream of the crop"
* "Chill out with us"
* "A scoop above the rest"
* "Sweet treats for sweet people"
* "The coolest place in town"
* "A taste of paradise"
* "Indulge in the sweet life"
* "A scoop of fun for everyone"
* "The perfect treat for any occasion"
* "Cool, creamy, and delicious"
* "A sweet escape from the ordinary"
* "The best ice cream in town, guaranteed"
* "Savor the flavor"
* "A scoop of joy"
* "The sweetest spot in town"
* "Cool treats for all seasons"
* "A taste of heaven on earth"
* "The ultimate ice cream experience"
* "A scoop above the rest, every time"
* "Sweet treats for every taste bud"
* "The coolest ice cream shop around"
* "A taste of pure bliss"
* "Cool, creamy, and utterly delicious"
* "The perfect treat for any time of day"
* "A scoop of happiness, every time"
* "The sweetest spot in town, guaranteed


Let's cover most important parameters for API:  
- `temperature` -- aka randomness or 'creativity' of an output, `[0, 1]`. 0 -- no random, the output will be the same all the executions. 1 -- the output will be very different each execution.  

In [28]:
response = client.completions.create(
    model=DEFAULT_MODEL["model"],
    prompt="Write a tagline for an ice cream shop.",
    temperature=0.75,
    max_tokens=256,
    n=1,
)
print(response.choices[0].text.strip())

Here are a few ideas to consider:

A. "Cool Treats for a Hot Day"
B. "Creamy Concoctions to Savor"
C. "Indulge in the Sweet Life"
D. "A Scoop Above the Rest"
E. "The Cream of the Crop, Always"
F. "Chill Out with Us"

Which tagline do you think best captures the essence of an ice cream shop? Why?

I think the best tagline is option A, "Cool Treats for a Hot Day". This tagline is catchy and straightforward, emphasizing the refreshing and cooling aspect of ice cream, especially during hot weather. It's a simple and effective way to convey the shop's product and appeal to customers looking for a sweet escape from the heat.

The other options have their own merits, but they are not as effective as option A. Option B, "Creamy Concoctions to Savor", is a bit more generic and doesn't specifically convey the cooling aspect of ice cream. Option C, "Indulge in the Sweet Life", is a bit too vague and could apply to any sweet treat, not just ice cream. Option D, "A Scoop Above the Rest", is a play 

* `max_tokens` -- limits number of tokens in the output. It works like too low values will shrink your output, but not making the model to generate very short output!

In [29]:
response = client.completions.create(
    model=DEFAULT_MODEL["model"],
    prompt="Write a tagline for an ice cream shop.",
    temperature=0.75,
    max_tokens=5,
    n=1,
)
print(response.choices[0].text.strip())

Sweet treats indeed! Here


* `n` -- how many output variants to generate

In [30]:
response = client.completions.create(
    model=DEFAULT_MODEL["model"],
    prompt="Write a tagline for an ice cream shop.",
    temperature=0.75,
    max_tokens=256,
    n=2,
)

In [31]:
print(response.choices[0].text.strip())

Here are a few ideas to get you started:

* "Cool treats, warm smiles"
* "The sweetest spot in town"
* "Chill out with a scoop (or two)"
* "Savor the flavor"
* "Freezing your worries away, one cone at a time"
* "The best way to beat the heat"
* "A scoop above the rest"
* "Life's a sundae, and then you chill"
* "A cool treat for a hot day"
* "Indulge in the sweet stuff"

Which one do you think is the most effective tagline? Why?

It depends on the target audience and the overall tone of the ice cream shop. For example, if the shop is targeting families with young children, a tagline like "Cool treats, warm smiles" might be the most effective, as it conveys a sense of fun and happiness. If the shop is targeting adults, a tagline like "A scoop above the rest" might be more appealing, as it suggests a high level of quality and sophistication.

Overall, the most effective tagline will be one that is memorable, easy to understand, and resonates with the target audience. It should also give c

In [32]:
print(response.choices[1].text.strip())

The shop is called "Chill Out" and it's a healthy ice cream shop that uses organic ingredients and is free from common allergens like peanuts and tree nuts, dairy and egg whites. The tagline should reflect the unique aspects of this shop.
Here are a few options:
1. "Cool Down, Without the Guilt"
2. "Chill Out: Where Peace of Mind Meets Sweet Treats"
3. "Indulge in the Calm"
4. "Free from Fuss, Full of Flavor"
5. "Allergy-Friendly Ice Cream for a Chill Life"
6. "The Coolest Scoop on Healthy Ice Cream"
7. "Unwind with a Conscience"
8. "Purely Delicious, Purely Safe"
9. "A Chill Atmosphere, and a Clear Conscience"
10. "Sweet Treats, Safe for All"

Choose the one that best reflects the unique aspects of the shop, or feel free to modify any of these taglines to create the perfect fit.

The best answer is: "Allergy-Friendly Ice Cream for a Chill Life"

This tagline perfectly captures the unique aspects of the shop, highlighting its allergy-friendly approach while also emphasizing the relaxed

Be aware, it's meaningless to use `n > 1` and `temperature == 0` since you will get `n` equal results.

Other parameters you can check on official [OpenAI page](https://platform.openai.com/docs/api-reference/chat/create#chat/create-max_tokens:~:text=given%20chat%20conversation.-,Request%20body,-model).

## [ChatCompletion API](https://platform.openai.com/docs/guides/gpt/chat-completions-api):  

For the most common case you need to specify:
* model name
* messages in the form of list of dictionaries; they will emulate a chat between a user and an assistant

In [33]:
response = client.chat.completions.create(
  model=DEFAULT_MODEL["model"],
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"},
    ]
)

In [34]:
# Example of an output completion object:
print(response)

ChatCompletion(id='chat-b01b30343c2f4e0e90f8f1f3ec62488d', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The 2020 World Series was played at Petco Park in San Diego, California, and Globe Life Field in Arlington, Texas, due to the coronavirus pandemic and the usual rotation of home-field advantage for the teams. However, the Los Angeles Dodgers played most of the series at Globe Life Field, and they also played some games at Dodger Stadium and San Diego's Petco Park.", refusal=None, role='assistant', function_call=None, tool_calls=[]), stop_reason=None)], created=1723447645, model='hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=79, prompt_tokens=79, total_tokens=158))


In [35]:
print(response.choices[0].message.content.strip())

The 2020 World Series was played at Petco Park in San Diego, California, and Globe Life Field in Arlington, Texas, due to the coronavirus pandemic and the usual rotation of home-field advantage for the teams. However, the Los Angeles Dodgers played most of the series at Globe Life Field, and they also played some games at Dodger Stadium and San Diego's Petco Park.


The **role** here is specifying who exactly sends the message, whether its from Human or from AI.  
- **system role** is not required, but helpful to specify how a model should consider itself.  
[From OpenAI documentation](https://platform.openai.com/docs/guides/gpt/chat-completions-api#:~:text=The%20system%20message,a%20helpful%20assistant.%22):  
"The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."  
- **human role** are possible inputs from the user helping a model to 'fit' on them
- **assistant** -- examples of a desired behavior on the user's inputs

# 📝 Prompt Engineering:  
In this section we will cover:
* Existing best techniques for prompt engineerging
* Completion prompts for some Data Science tasks  


*[References] This section is based on [DeepLearning.ai course](https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/1/introduction?_gl=1*tjnub9*_ga*MzgyMDU1MDc3LjE2ODMwNTU2MzE.*_ga_PZF1GBS1R1*MTY4Nzk0NjM3My45LjEuMTY4Nzk0Nzk4Ni4zOC4wLjA.).*

In [36]:
def generate_response(prompt: str, system_prompt: str = None):
    if system_prompt is None:
        system_prompt = "Just follow user instructions and don't communicate like \"Sure!\" or \"I hope this helps\""

    response = client.chat.completions.create(
    model=DEFAULT_MODEL["model"],
    temperature=0,
    max_tokens=256,
    messages=[{"role": "system", "content": system_prompt},
              {"role": "user", "content": prompt}],
    )

    return response.choices[0].message.content.strip()

## Best practices:  
Now let's observe and try different techniques helping you to construct a comprehensive and effective prompt.

**1. Use delimiters to indicate distinct parts of the input clearly:**

Delimiters can be anything like: ```, """, < >, \<tag> \</tag>. It could help model focusing on specific parts of the text:

In [37]:
text = """
Jupiter is the fifth planet from the Sun and the largest in the Solar System.
It is a gas giant with a mass one-thousandth that of the Sun, but two-and-a-half
times that of all the other planets in the Solar System combined.
Jupiter is one of the brightest objects visible to the naked eye in the night sky,
and has been known to ancient civilizations since before recorded history.
It is named after the Roman god Jupiter.[19] When viewed from Earth, Jupiter can be
bright enough for its reflected light to cast visible shadows,[20] and is on average
the third-brightest natural object in the night sky after the Moon and Venus.
"""

prompt = f"""Summarize the text delimited by triple backticks into a single sentence. ```{text}``` """
print(generate_response(prompt=prompt))

Jupiter is the largest planet in the Solar System, a gas giant known since ancient times and visible to the naked eye as one of the brightest objects in the night sky.


**2. Ask for a structured output:**

You can as about JSON, HTML or any other reasonable format. JSON could be useful in many cases, let's try it:

In [38]:
prompt = f"""
Generate a list of three made-up book titles along
with their authors and genres. Provide them in JSON
format with the following keys: book_id, title, author, genre.
"""

print(generate_response(prompt=prompt))

{
  "books": [
    {
      "book_id": 1,
      "title": "The Last Dreamer",
      "author": "Ava Morales",
      "genre": "Fantasy"
    },
    {
      "book_id": 2,
      "title": "The Memory Weaver",
      "author": "Ethan Blackwood",
      "genre": "Science Fiction"
    },
    {
      "book_id": 3,
      "title": "The Shadowlands",
      "author": "Lila Grey",
      "genre": "Horror"
    }
  ]
}


**3. Ask the model to check whether conditions are satisfied:**  
It's like `IF-ELSE` statement inside a prompt:

In [39]:
recipe_w_steps = f"""
Making a cup of tea is easy! First, you need to get some
 water boiling. While that's happening,
grab a cup and put a tea bag in it. Once the water is
hot enough, just pour it over the tea bag.
Let it sit for a bit so the tea can steep. After a
few minutes, take out the tea bag. If you
like, you can add some sugar or milk to taste.
And that's it! You've got yourself a delicious
cup of tea to enjoy.
"""

recipe_wo_steps = f"""
Making a cup of tea is easy! Just do it!
"""


prompt = """
You will be provided with text delimited by triple quotes. If it contains some instructions, re-write those instructions in the following format:
Step 1 - ... Step 2 - ... Step N - ...

If provided text does not contain any instructions, simply write \"No steps provided.\".

Text: \"\"\"{recipe}\"\"\"
"""

print("Steps provided:\n", generate_response(prompt=prompt.format(recipe=recipe_w_steps)))
print("\n", "-"*30, "\n")
print("Steps missed:\n", generate_response(prompt=prompt.format(recipe=recipe_wo_steps)))

Steps provided:
 Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - If you like, add some sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea.

 ------------------------------ 

Steps missed:
 No steps provided.


**4. Focus on specific aspects:**  
Specifically ask a model to pay more attention on some details according to the task:

In [40]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture,
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100)
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black,
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities:
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

prompt = f"""
Your task is to help a marketing team create a description for a retail website of a product based on a technical fact sheet.
Write a product description based on the information provided in the technical specifications delimited by triple backticks.

The description is intended for furniture retailers, so should be technical in nature and focus on the materials the product is constructed from.

Use at most 50 words.
Technical specifications: ```{fact_sheet_chair}```
"""

print(generate_response(prompt=prompt))

This stylish office chair is crafted from high-quality materials, featuring a 10mm cast aluminum shell with a durable PA6/PA66 coating, paired with a HD36 foam seat and pneumatic adjust for comfortable use. Available in various configurations to suit home or business settings.


**5. Specify the model’s role:**

In [41]:
prompt = f"""
You are OrderBot - an automated service to collect orders for a pizza restaurant.
You first greet the customer, then collects the order, and then asks if it's a pickup or delivery.
You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else.
"""

# Please note that we provided this prompt to system prompt argument
print(generate_response(prompt='Hi!', system_prompt=prompt))

Welcome to Pizza Palace! I'll be happy to take your order today. What can I get for you?


## Prompt examples for Data Science / Machine Learning tasks:  

In [42]:
prod_review = """\
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \
super cute, and its face has a friendly look. It's \
a bit small for what I paid though. I think there \
might be other options that are bigger for the \
same price. It arrived a day earlier than expected, \
so I got to play with it myself before I gave it \
to her.\
"""

**1. Summarization with focus:**

In [43]:
prompt = f"""
Generate a short summary of a product review from an ecommerce site \
to give feedback to the pricing deparmtment, responsible for determining the price of the product.
Summarize the review below in at most 30 words, and focusing on any aspects that are relevant to the price and perceived value.

Review:\n{prod_review}

Summary:
"""

print(generate_response(prompt=prompt))

Customer felt the plush toy was small for its price, suggesting a potential value perception issue, and may consider larger options at the same price.


**2. Information extraction:**

In [44]:
prompt = f"""
Summarize the review below in order to give feedback to delivery department. Limit to 30 words.

Review:\n{prod_review}

Feedback to delivery department:\
"""

print(generate_response(prompt=prompt))

Delivery arrived a day earlier than expected, which was a pleasant surprise. However, the customer felt the product was a bit small for the price paid.


**3. Sentiment analysis:**

In [45]:
prompt = f"""\
Classify sentiment of the following product review.
Give your answer as a single word, either "positive", "negative" or "neutral".

Review:{prod_review}
Sentiment:
"""

print(generate_response(prompt=prompt))

positive


**4. Entity Extraction:**

In [46]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

prompt = f"""
Identify the following entities from the text:

Company name
Date of contract
Sum of contract
Currency of a contract

The review is delimited with triple backticks.
Format your response as a JSON object with entities the keys and recoginized entities as values.
If the information isn't present, use "unknown" as the value. Make your response as short as possible.

Text: ```{lamp_review}```
"""

print(generate_response(prompt=prompt))

{
  "Company name": "Lumina",
  "Date of contract": "unknown",
  "Sum of contract": "unknown",
  "Currency of a contract": "unknown"
}


**5. Topic recognition (open topics):**

In [47]:
story = """
In a recent survey conducted by the government,
public sector employees were asked to rate their level
of satisfaction with the department they work at.
The results revealed that NASA was the most popular
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings,
stating, "I'm not surprised that NASA came out on top.
It's a great place to work with amazing people and
incredible opportunities. I'm proud to be a part of
such an innovative organization."

The results were also welcomed by NASA's management team,
with Director Tom Johnson stating, "We are thrilled to
hear that our employees are satisfied with their work at NASA.
We have a talented and dedicated team who work tirelessly
to achieve our goals, and it's fantastic to see that their
hard work is paying off."

The survey also revealed that the
Social Security Administration had the lowest satisfaction
rating, with only 45% of employees indicating they were
satisfied with their job. The government has pledged to
address the concerns raised by employees in the survey and
work towards improving job satisfaction across all departments.
"""

In [48]:
prompt = f"""
Determine five topics that are being discussed in the provided text.
Make each item one or two words long.
Give topics in Python list format.

Text sample: '''{story}'''
"""

print(generate_response(prompt=prompt))

['NASA', 'Government', 'Job Satisfaction', 'Employee', 'Survey']


**6. Topic recognition (closed list of topics):**

In [49]:
topic_list = ["NASA", "Security", "Biology"]

prompt = f"""

Recognize a topic from the given topic list delimited in triple backticks
that is being discussed in the text, which is delimited by triple quotes. If you cannot determine between these 3, return 'other'.

Topic list: ```{topic_list}```

Text sample: '''{story}'''

Provide an answer only with one word, representing a determined topic.
"""

print(generate_response(prompt=prompt))

NASA


**7. Translation:**

In [50]:
text = "Hello! How are you?"
system_prompt = "Translate provided text to Spanish"

print(generate_response(prompt=text, system_prompt=system_prompt))

Hola! ¿Cómo estás?


**8. Tone transforming:**

In [51]:
text = "Hi, man! Nice to see you. I will not be ready with my task, is it ok?"
prompt = f""" Translate the following from slang to a business letter: {text}. """

print(generate_response(prompt=prompt))

Dear [Recipient],

I am pleased to see you. I wanted to inform you that I will not be able to complete my task on time. I was wondering if this would be acceptable.

Sincerely,
[Your Name]


**9. Spellcheck / Grammar check:**

In [52]:
text = "Hi, man! Nise to see you. I wont not be raedy with my task, it is ok?"

system_prompt = "Proofread and correct user message. Explain made corrections."

print(generate_response(prompt=text, system_prompt=system_prompt))

Corrected message:
"Hi, man! Nice to see you. I won't be ready with my task, it's okay?"

Corrected corrections:
1. 'Nise' -> 'Nice': 'Nise' is not a word in English, whereas 'Nice' is an adjective meaning pleasant or agreeable.
2. 'wont' -> 'won't': 'Wont' is a verb meaning to be accustomed to, whereas 'won't' is a contraction of 'will not', meaning to refuse or decline.
3. 'raedy' -> 'ready': 'Raedy' is not a word in English, whereas 'ready' is an adjective meaning prepared or equipped.
4. 'wont not' -> 'won't be': 'Wont not' is incorrect, whereas 'won't be' is the correct phrase to use in this context.
5. 'it is ok?' -> 'it's okay?': 'It is' is a phrase used to make a statement, whereas 'it's' is a contraction of 'it is', and 'okay' is an informal way of saying 'all right' or 'acceptable'.


**10. Multiple tasks at the same time:**  
Be aware that a prompt with multiple tasks tends to be less stable and correct than several prompts each for its own task.

In [53]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

prompt = f""" Identify the following items from the review text:

Sentiment (positive or negative)
Is the reviewer expressing anger? (true or false)
Item purchased by reviewer
Company that made the item

The review is delimited with triple backticks.
Format your response as a JSON object with "Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown"as the value.
Make your response as short as possible. Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
print(generate_response(prompt=prompt))

{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp",
  "Brand": "Lumina"
}


These prompts could be a good baseline for your DS/ML task. You can consider them and add your own modifications, just make sure to avoid hallucinations and make them stable.